In [1]:
#import packages
import tensorflow as tf
import numpy as np
import random
import functools
from rl.memory import Memory

In [8]:
class DQfDAgent:
    def __init__(self, env, configuration,trajectories =None):
        self.session = tf.InteractiveSession()
        self.conf = configuration
        
        
        self.replay = Memory(capacity=self.conf.replay_buffer_size, permanent_data=len(trajectories))
        self.demos = Memory(capacity=self.conf.demo_buffer_size, permanent_data=self.conf.demo_buffer_size)
        self.include_trajectories(trajectories= trajectories)  # add demo data to both demo_memory & replay_memory
        self.step = 0
        self.eps = self.conf.INITIAL_EPSILON
        self.state_no = env.observation_space.shape[0]
        self.action_no = env.action_space.n
        
        self.action_b 
        self.y_input
        self.wweights 
        self.n_step_y_input 
        self.isdemo 
        self.eval_input
        self.select_input

        self.q_evaluation
        self.q_choice

        self.loss
        self.optimizer
        self.update
        self.ae

        self.save = tf.train.Saver()

        self.session.run(tf.global_variables_initializer())
      
    #function to include transitions to demonstration memory
    def include_tranjectories(self, trajectories):
        for tra in trajectories :
            self.demos.store(np.array(tra, dtype=object))
            self.replay.store(np.array(tra, dtype=object))
            assert len(tra) == 10

#function for greedy policy
    def greedy_act(self, current_state):
        if np.random.random() < self.epsilon:
            return np.random.randint(0, self.action_no - 1)
        return np.argmax(self.Q_select.eval([current_state])[0])
    
#function for training 
    def train_ahead(self):
        for tr in range(self.conf.PRETRAIN_STEPS):
            self.q_train(train=True)
            if tr % 200 == 0 and tr > 0:
                print('Training step with expert demonsrtations: {}'.format(tr))
        self.step= 0
        print('Finished demo training')

#function for deep neural network layers
    def neural_net_layers(self, current_state, cols, no_units1, no_units2, input_weight, input_bias, reg=None):
        a_n = self.action_no
        with tf.variable_scope('layer1'):
            weight1 = tf.get_variable('weight1', [a_n, no_units1], initializer=input_weight, collections=cols, regularizer=reg)
            bias1 = tf.get_variable('bias1', [1, no_units1], initializer=input_bias, collections=cols, regularizer=reg)
            dense1 = tf.nn.relu(tf.matmul(current_state, weight1) + bias1)
        with tf.variable_scope('layer2'):
            weight2 = tf.get_variable('weight2', [no_units1, no_units2], initializer=input_weight, collections=cols, regularizer=reg)
            bias2 = tf.get_variable('bias2', [1, no_units2], initializer=input_bias, collections=cols, regularizer=reg)
            dense2 = tf.nn.relu(tf.matmul(dense1, weight2) + bias2)
        with tf.variable_scope('layer3'):
            weight3 = tf.get_variable('weight3', [no_units2, a_n], initializer=input_weight, collections=cols, regularizer=reg)
            bias3 = tf.get_variable('bias3', [1, a_n], initializer=input_bias, collections=cols, regularizer=reg)
            dense3 = tf.matmul(dense2, weight3) + bias3
        return dense3

    #function for selecting network
    def select_q(self):
        with tf.variable_scope('Network_selection') as scope:
            cols = ['network_params_select', tf.GraphKeys.GLOBAL_VARIABLES]
            input_weight = tf.random_uniform_initializer(-0.1, 0.1)
            input_bias = tf.constant_initializer(0.1)
            reg = tf.contrib.layers.l2_regularizer(scale=0.2)  
            return self.build_layers(self.select_input, cols, 24, 24, input_weight, input_bias, reg)
       
    #function for evaluating the network 
    def eval_q(self):
        with tf.variable_scope('Network_evaluation') as scope:
            cols = ['network_params_evaluate', tf.GraphKeys.GLOBAL_VARIABLES]
            input_weight = tf.random_uniform_initializer(-0.1, 0.1)
            input_bias = tf.constant_initializer(0.1)
            return self.build_layers(self.eval_input, cols, 24, 24, input_weight, input_bias)
        

    def loss_lin(self, ae, a):
        return 0.0 if ae == a else 0.75

    def loss_selec(self, select_q):
        inp = 0.0
        for i in range(self.conf.BATCH_SIZE):
            ae = self.action_batch[i]
            max_val = float("-inf")
            for act in range(self.action_n):
                max_val = tf.max(select_q[i][act] + self.loss_lin(ae, act), max_val)
            inp += self.isdemo[i] * (max_val - select_q[i][ae])
        return inp

   
    def loss(self):
        loss_dq = tf.reduce_mean(tf.squared_difference(self.select_q, self.y_input))
        lloss_dq = tf.reduce_mean(tf.squared_difference(self.select_q, self.n_step_y_input))
        loss_inp = self.loss_selec(self.select_q)
        loss_l2 = tf.reduce_sum([tf.reduce_mean(reg_l) for reg_l in tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)])
        return self.weights * tf.reduce_sum([l * lam for l, lam in zip([loss_dq, lloss_dq, loss_inp, loss_l2], self.conf.LAMBDA)])


    def abs_er(self):
        return tf.reduce_sum(tf.abs(self.y_input - self.select_q), axis=1)  

 
    def optimization(self):
        optimizer = tf.train.AdamOptimizer(self.conf.LEARNING_RATE)
        return optimizer.minimize(self.loss)  
    

    def update_target(self):
        select_p = tf.get_collection('network_params_select')
        eval_p = tf.get_collection('network_params_evaluate')
        return [tf.assign(e, s) for e, s in zip(eval_p, select_p)]

#     def save_model(self):
#         print("Model saved in : {}".format(self.saver.save(self.sess, self.config.MODEL_PATH)))

#     def restore_model(self):
#         self.saver.restore(self.sess, self.config.MODEL_PATH)
#         print("Model restored.")

    def perceive(self, trajectory):
        self.replay.store(np.array(trajectory))
        
        if self.replay.full():
            self.epsilon = max(self.conf.FINAL_EPSILON, self.epsilon * self.conf.EPSILIN_DECAY)

    def train_nework(self, train =False, update=True):
       
        if self.replay.full() or train==True:
            return 
        self.step= self.step+ 1

        assert self.replay.full() or train

        actual_mem = self.demos if train else self.replay
        tree_idx, minibatch, wweights = actual_mem.sample(self.conf.BATCH_SIZE)

        np.random.shuffle(minibatch)
        current_state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        new_state_batch = [data[3] for data in minibatch]
        done_batch = [data[4] for data in minibatch]
        demos_data = [data[5] for data in minibatch]
        nth_step_reward_batch = [data[6] for data in minibatch]
        nth_step_state_batch = [data[7] for data in minibatch]
        nth_step_done_batch = [data[8] for data in minibatch]
        actual_no = [data[9] for data in minibatch]

        # provide for placeholder，compute first
        select_q = self.select_q.eval(new_state_batch)
        eval_q = self.eval_q.eval(new_state_batch)
        n_step_select_q = self.select_q.eval(nth_step_state_batch)
        n_step_eval_q = self.eval_q.eval(nth_step_state_batch)

        y_batch = np.zeros((self.conf.BATCH_SIZE, self.action_no))
        n_step_y_batch = np.zeros((self.conf.BATCH_SIZE, self.action_no))
        for i in range(self.config.BATCH_SIZE):

            temp = self.select_q.eval( current_state_batch[i].reshape((-1, self.state_dim)))[0]
            temp_0 = np.copy(temp)
            # add 1-step reward
            action = np.argmax(select_q[i])
            temp[action_batch[i]] = reward_batch[i] + (1 - int(done_batch[i])) * self.conf.GAMMA * eval_q[i][action]
            y_batch[i] = temp
            # add n-step reward
            action = np.argmax(n_step_select_q[i])
            q_nth_step = (1 - int(nth_step_done_batch[i])) * self.config.GAMMA**actual_no[i] * n_step_eval_q[i][action]
            temp_0[action_batch[i]] = nth_step_reward_batch[i] + q_nth_step
            n_step_y_batch[i] = temp_0

        _, abs_errors = self.session.run([self.optimize, self.abs_errors],
                                      y_batch,
                                      n_step_y_batch,
                                      current_state_batch,
                                      action_batch,
                                      demos_data,
                                      wweights)


     
        if update and self.step % self.conf.UPDATE_TARGET_NET == 0:
            self.session.run(self.update_target_net)
    
    
    
    
    